In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

--2025-06-16 20:53:14--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5820 (5.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   5.68K  --.-KB/s    in 0s      

2025-06-16 20:53:15 (62.7 MB/s) - ‘minsearch.py.1’ saved [5820/5820]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = 'the course has already started, can I still join?'

In [9]:
index.fit(documents)

In [10]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [12]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [13]:
import groq

In [14]:
from groq import Client

In [15]:
client = Client(api_key=os.getenv("GROQ_API_KEY"))

In [16]:
q

'the course has already started, can I still join?'

In [35]:
response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
      {
        "role": "user",
        "content": q
      }
]
)

response.choices[0].message.content

"It's best to check with the course instructor, professor, or the institution's administration to see if it's still possible to join the course. Some courses may have a flexible enrollment period, while others might have already started and have strict attendance or participation requirements.\n\nHere are a few possible scenarios:\n\n1. **The course is still open for enrollment**: The instructor or institution might allow you to join the course, but you may need to catch up on any material you've missed so far.\n2. **The course has a waiting list**: If the course is full, you might be able to join a waiting list in case a spot becomes available.\n3. **The course has strict attendance requirements**: If you've missed the first class or sessions, you might not be able to join the course, especially if attendance is mandatory.\n4. **The course has already covered too much material**: Depending on the course structure and content, it might be challenging to join mid-course, especially if y

In [36]:
prompt_template = """

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT deosn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}

""".strip()

In [37]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [38]:
prompt = prompt_template.format(question=q, context=context).strip()

In [39]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT deosn't contain the answer, output NONE.

QUESTION: the course has already started, can I still join?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only

In [41]:
response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
      {
        "role": "user",
        "content": prompt
      }
]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."